<a href="https://colab.research.google.com/github/mnvlucian/Google-Earth-Engine-Climogram/blob/main/Climograma_Lucian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code created using the following videos as a basis: 
https://www.youtube.com/watch?v=hsJxeecHqeM and 
https://www.youtube.com/watch?v=AZNS7LQ78UY

In [ ]:
!pip install geopandas
!pip install earthengine-api
!pip install ipygee

     |████████████████████████████████| 1.0 MB 7.4 MB/s 
     |████████████████████████████████| 6.3 MB 40.2 MB/s 
     |████████████████████████████████| 16.7 MB 315 kB/s 
     |████████████████████████████████| 3.3 MB 12.9 MB/s 
     |████████████████████████████████| 129 kB 43.2 MB/s 
     |████████████████████████████████| 74 kB 1.5 MB/s 
     |████████████████████████████████| 44 kB 2.1 MB/s 
  Created wheel for ipygee: filename=ipygee-0.0.17-py3-none-any.whl size=40379 sha256=ff71b44ca05786d666089bb57ee4d43315c504edffea1145133d4f636fa8b4b6
  Stored in directory: /root/.cache/pip/wheels/73/af/7e/a9d3fac7d32e649c63fdbdb9b320181b20eb4bfdc80af13974
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92107 sha256=a4f8847a165f80a8805748da2fa2b263c5193317ab509dad444e160f56de06b7
  Stored in directory: /root/.cache/pip/wheels/5c/55/29/0a09dcf6b39bba8c890e40171cac3e3607a5c4b354b7e9447c
Successfully built ipygee geetools


In [ ]:
#import libs
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import json
import ee
%matplotlib inline
import seaborn as sns
from ipygee import*
import altair as alt
from shapely.geometry import Polygon

In [ ]:
# GEE Authentication
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=xB9HDA-YJyK1GKjl3KvhZ92CnttTuL9Zy8zedN20iV8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjkLIRPy9PmTOh-vFBka99C3o7lRTM5Wa9UpG3t5mBgVaMuTzN7RaI

Successfully saved authorization token.


In [ ]:
#@title Default title text
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# read the shapefile of the study area
area = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/VPLN_cities.shp')

# converting from shapefile to json (aoi = area of interesting)
aoi = area.to_json()
# load json file
aoi = json.loads(aoi)
# select features
aoi = aoi['features']

aoi = ee.FeatureCollection(aoi).geometry()

# Due to the RAM memory limit in EE virtual machine, it's recommended 
# to generate 5 years for each execution
# "endyear" defines the time range, but itself isn't included, i.e. 
# startyear1 >= 2000 and endyear1<2005
startyear1 = 2000
endyear1 = 2005

# Set date in ee date format
startdate1 = ee.Date.fromYMD(startyear1,1,1)
enddate1 = ee.Date.fromYMD(endyear1,12,31)

# Make a list for years
years1 = range(startyear1, endyear1);

# Make a list for months
months = range(1,13)

# Set date in ee date format
start_date1 = ee.Date.fromYMD(startyear1,1,1)
end_date1 = ee.Date.fromYMD(endyear1,12,31)

# Importing precipitation data
CHIRPS1 = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
                            .select('precipitation')\
                            .filterDate(start_date1,end_date1)\
                            .filterBounds(aoi)\

# Defining dates lists
dates_chirps1 = CHIRPS1.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-DD'))
print(dates_chirps1.getInfo())

# Defining dates lists
def date (image):
  return image.clip(aoi).copyProperties(image, ['system:time_start']).set('date', image.date().format())

CHIRPS1 = CHIRPS1.map(date)
CHIRPS1.first().propertyNames().getInfo()

#  Calculating Monthly Sum
def MonthlySum1(imageCollection1):
  mylist1 = ee.List([])
  for y in years1:
    for m in months:
      w = imageCollection1.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month')).sum()
      mylist1 = mylist1.add(w.set('year', y)\
                          .set('month', m)\
                          .set('date', ee.Date.fromYMD(y,m,1)))
  
  return ee.ImageCollection.fromImages(mylist1).set('date', ee.Date.fromYMD(y, m, 1))\

# run the calcMonthlyMean function
monthlyCHIRPS = ee.ImageCollection(MonthlySum1(CHIRPS1))
monthlyCHIRPS.first().propertyNames().getInfo()

print("First from CHIRPS")
print(monthlyCHIRPS.first().getInfo())
# print("All from CHIRPS")
# print(monthlyCHIRPS.getInfo())

['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-32', '2015-02-33', '2015-02-34', '2015-02-35', '2015-02-36', '2015-02-37', '2015-02-38', '2015-02-39', '2015-02-40', '2015-02-41', '2015-02-42', '2015-02-43', '2015-02-44', '2015-02-45', '2015-02-46', '2015-02-47', '2015-02-48', '2015-02-49', '2015-02-50', '2015-02-51', '2015-02-52', '2015-02-53', '2015-02-54', '2015-02-55', '2015-02-56', '2015-02-57', '2015-02-58', '2015-02-59', '2015-03-60', '2015-03-61', '2015-03-62', '2015-03-63', '2015-03-64', '2015-03-65', '2015-03-66', '2015-03-67', '2015-03-68', '2015-03-69', '2015-03-70', '2015-03-71', '2015

In [ ]:
# Statistics function

def reduce (image):
  serie_reduce = image.reduceRegions(**{
      'collection': aoi,
      'reducer': ee.Reducer.mean().combine(**{
      'reducer2': ee.Reducer.max(),
      'sharedInputs': True}).combine(**{
        'reducer2': ee.Reducer.min(),
        'sharedInputs': True}),
      'scale': 1000
      })
  
  serie_reduce = serie_reduce.map(lambda f: f.set({'millis': image.get('millis')}))\
                             .map(lambda f: f.set({'date': image.get('date')}))\
                             .map(lambda f: f.set({'month': image.get('month')}))\
                             .map(lambda f: f.set({'year': image.get('year')}))
  return serie_reduce.copyProperties(image, ['system:time_start'])

precipitation_reduce = monthlyCHIRPS.map(reduce)\
                                    .flatten()\
                                    .sort('date', True)\
                                    .select(['millis', 'date', 'max', 'mean', 'min', 'month', 'year'])

print("Size:",precipitation_reduce.size().getInfo())
# print("First:",precipitation_reduce.first().getInfo())
# Checking the data previously generated:
# df_debug = pd.DataFrame(precipitation_reduce.first().getInfo())
# debug = df_debug.to_csv('/content/drive/MyDrive/Colab Notebooks/precipitation_reduce.csv')

Size: 60


In [ ]:
# Checking the data previously generated:
# df_debug = pd.DataFrame(precipitation_reduce.first().getInfo())
# debug = df_debug.to_csv('/content/drive/MyDrive/Colab Notebooks/precipitation_reduce.csv')

# creating the data list
# prec_list = precipitation_reduce.reduceColumns(ee.Reducer.toList(3), ['date', 'month', 'year']).values().get(0)
prec_list = precipitation_reduce.reduceColumns(ee.Reducer.toList(6), ['date', 'max', 'mean', 'min', 'month', 'year']).values().get(0)
print("Precipitation reduce columns:",prec_list.getInfo())

# df_debug = pd.DataFrame(prec_list.getInfo())
# debug = df_debug.to_csv('/content/drive/MyDrive/Colab Notebooks/precipitation_reduce.csv')

# creating the dataframe

df_prec = pd.DataFrame(prec_list.getInfo(), columns=['date', 'max_prec', 'mean_prec', 'min_prec', 'month', 'year'])
# print("First:",df_prec.getInfo())
df_prec

# export to .csv
prec_csv1 = df_prec.to_csv('/content/drive/MyDrive/Colab Notebooks/df_prec_2000-2005.csv')

Precipitation reduce columns: [[{'type': 'Date', 'value': 1420070400000}, 379.7691459655762, 251.29638466772676, 137.19823455810547, 1, 2015], [{'type': 'Date', 'value': 1422748800000}, 454.08578300476074, 287.33588991581615, 174.76961708068848, 2, 2015], [{'type': 'Date', 'value': 1425168000000}, 415.4996757507324, 235.19832111909292, 119.80703735351562, 3, 2015], [{'type': 'Date', 'value': 1427846400000}, 162.4901638031006, 72.1620464574701, 29.184499740600586, 4, 2015], [{'type': 'Date', 'value': 1430438400000}, 268.1154975891113, 90.82484114489692, 22.905876994132996, 5, 2015], [{'type': 'Date', 'value': 1433116800000}, 94.24646377563477, 24.951499640417243, 9.551414251327515, 6, 2015], [{'type': 'Date', 'value': 1435708800000}, 226.70220947265625, 51.777678044535385, 10.50576376914978, 7, 2015], [{'type': 'Date', 'value': 1438387200000}, 69.50458812713623, 23.612201317061054, 7.416811227798462, 8, 2015], [{'type': 'Date', 'value': 1441065600000}, 199.5310869216919, 74.118872414288